In [1]:
import pandas as pd
import numpy as np
import os
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [2]:
from flask_sqlalchemy import SQLAlchemy
from flask import (
    Flask,
    render_template,
    jsonify,
    request,
    redirect)
from sqlalchemy.orm import Session

app = Flask(__name__)

app.config['SQLALCHEMY_DATABASE_URI'] = "sqlite:///belly_button_biodiversity.sqlite"
db = SQLAlchemy(app)

C:\Users\dan.brueckman\Anaconda3\envs\PythonData\lib\site-packages\flask_sqlalchemy\__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [3]:
engine = create_engine("sqlite:///belly_button_biodiversity.sqlite")

In [4]:
engine.table_names()

['otu', 'samples', 'samples_metadata']

In [5]:
Base = automap_base()
Base.prepare(engine, reflect=True)
session = Session(engine)

In [6]:
OTU = Base.classes.otu
Samples = Base.classes.samples
MD = Base.classes.samples_metadata

In [7]:
@app.route("/")
def home():
    return render_template("index.html")

In [8]:
@app.route("/names")
def names():
    
    """List of sample names.

    Returns a list of sample names in the format
    [
        "940",
        "941",
        "943",
        "944",
        "945",
        "946",
        "947",
        ...
    ]

    """
    
    results = db.session.query(MD.SAMPLEID).all()
    results = list(np.ravel(results))
    return results

In [9]:
@app.route("/otu")
def otu:
    
    """List of OTU descriptions.

    Returns a list of OTU descriptions in the following format

    [
        "Archaea;Euryarchaeota;Halobacteria;Halobacteriales;Halobacteriaceae;Halococcus",
        "Archaea;Euryarchaeota;Halobacteria;Halobacteriales;Halobacteriaceae;Halococcus",
        "Bacteria",
        "Bacteria",
        "Bacteria",
        ...
    ]
    """
    
    results = db.session.query(OTU.lowest_taxonomic_unit_found).all()
    df = pd.DataFrame(results, columns =["otu_description"])
    return jsonify(df.to_dict(orient="records"))

In [10]:
@app.route('/metadata/<int:sample>')
def meta(sample):
    
    """MetaData for a given sample.

    Args: Sample in the format: `940`

    Returns a json dictionary of sample metadata in the format

    {
        AGE: 24,
        BBTYPE: "I",
        ETHNICITY: "Caucasian",
        GENDER: "F",
        LOCATION: "Beaufort/NC",
        SAMPLEID: 940
    }
    """
    req = db.session.query(MD.SAMPLEID).all()
    ravel = list(np.ravel(req))
    if sample in ravel:
        results = db.session.query(MD.AGE, MD.BBTYPE, MD.ETHNICITY, MD.GENDER, MD.LOCATION, MD.SAMPLEID).\
            filter(MD.SAMPLEID == sample).all()
        df = pd.DataFrame(results, columns = ["AGE", "BBTYPE", "ETHNICITY", "GENDER", "LOCATION", "SAMPLEID"])
        return jsonify(df.to_dict(orient="records"))

In [11]:
@app.route('/wfreq/<int:sample>')
def wash(sample):
    """Weekly Washing Frequency as a number.

    Args: Sample in the format: `940`

    Returns an integer value for the weekly washing frequency `WFREQ`
    """
    req = db.session.query(MD.SAMPLEID).all()
    ravel = list(np.ravel(req))
    if sample in ravel:
        results = db.session.query(MD.WFREQ).\
            filter(MD.SAMPLEID == sample).all()
        return int(np.ravel(results))

In [12]:
@app.route('/samples/<int:sample>')
def values(sample):
    
    """OTU IDs and Sample Values for a given sample.

    Sort your Pandas DataFrame (OTU ID and Sample Value)
    in Descending Order by Sample Value

    Return a list of dictionaries containing sorted lists  for `otu_ids`
    and `sample_values`

    [
        {
            otu_ids: [
                1166,
                2858,
                481,
                ...
            ],
            sample_values: [
                163,
                126,
                113,
                ...
            ]
        }
    ]
    """
    req = db.session.query(MD.SAMPLEID).all()
    ravel = list(np.ravel(req))
    if sample in ravel:
        results = engine.execute("select otu_id, BB_%s from samples" % sample).fetchall()
        df = pd.DataFrame(results, columns = ["otu_ids", "sample_values"])
        df.sort_values(by="sample_values", ascending = False, inplace = True)
        new_dict = df.to_dict(orient="list")
        return new_dict

In [13]:
!jupyter nbconvert --to script 1.ipynb

[NbConvertApp] Converting notebook 1.ipynb to script
[NbConvertApp] Writing 4182 bytes to 1.py


In [14]:
def ab():
    results = db.session.query(MD.SAMPLEID).all()
    results = list(np.ravel(results))
    return results

In [28]:
results = db.session.query(MD.SAMPLEID).all()
results = list(np.ravel(results))
results = list(map(str, results))
results

['940',
 '941',
 '943',
 '944',
 '945',
 '946',
 '947',
 '948',
 '949',
 '950',
 '952',
 '953',
 '954',
 '955',
 '956',
 '958',
 '959',
 '960',
 '961',
 '962',
 '963',
 '964',
 '966',
 '967',
 '968',
 '969',
 '970',
 '971',
 '972',
 '973',
 '974',
 '975',
 '978',
 '1233',
 '1234',
 '1235',
 '1236',
 '1237',
 '1238',
 '1242',
 '1243',
 '1246',
 '1253',
 '1254',
 '1258',
 '1259',
 '1260',
 '1264',
 '1265',
 '1273',
 '1275',
 '1276',
 '1277',
 '1278',
 '1279',
 '1280',
 '1281',
 '1282',
 '1283',
 '1284',
 '1285',
 '1286',
 '1287',
 '1288',
 '1289',
 '1290',
 '1291',
 '1292',
 '1293',
 '1294',
 '1295',
 '1296',
 '1297',
 '1298',
 '1308',
 '1309',
 '1310',
 '1374',
 '1415',
 '1439',
 '1441',
 '1443',
 '1486',
 '1487',
 '1489',
 '1490',
 '1491',
 '1494',
 '1495',
 '1497',
 '1499',
 '1500',
 '1501',
 '1502',
 '1503',
 '1504',
 '1505',
 '1506',
 '1507',
 '1508',
 '1510',
 '1511',
 '1512',
 '1513',
 '1514',
 '1515',
 '1516',
 '1517',
 '1518',
 '1519',
 '1521',
 '1524',
 '1526',
 '1527',
 '1530'